### 🌡️ Nhiệt độ & độ ẩm

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `temp`   | Nhiệt độ không khí trung bình tại bề mặt | °C |
| `dwpt`   | **Điểm sương (dew point)** – nhiệt độ tại đó hơi nước bắt đầu ngưng tụ (có mây, sương) | °C |
| `rhum`   | **Độ ẩm tương đối (relative humidity)** – phần trăm hơi nước trong không khí so với tối đa có thể chứa được | % |

📌 *Điểm sương và độ ẩm thường dùng để đánh giá mức độ oi bức, dễ tạo mây hoặc mưa.*

---

### 🌧️ Mưa, tuyết, nắng

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `prcp`   | **Lượng mưa (precipitation)** – tổng lượng mưa trong ngày | mm |
| `snow`   | Độ dày lớp tuyết (nếu có tuyết) | mm |
| `tsun`   | Thời lượng ánh nắng mặt trời trong ngày | phút |

---

### 🌬️ Gió & áp suất

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `wdir`   | **Hướng gió trung bình**, tính theo độ (0° là Bắc, 90° là Đông, 180° là Nam, 270° là Tây) | ° |
| `wspd`   | **Tốc độ gió trung bình** trong ngày | km/h |
| `wpgt`   | **Tốc độ gió lớn nhất (giật gió)** đo được trong ngày | km/h |
| `pres`   | Áp suất khí quyển trung bình | hPa (hectopascal) |

📌 *Áp suất khí quyển và gió là những yếu tố ảnh hưởng mạnh đến sự hình thành bão, mây, và thời tiết cực đoan.*

---

### ☁️ Mây

| Tên biến | Giải thích | Đơn vị |
|----------|------------|--------|
| `coco`   | **Mức độ che phủ mây (cloud cover)** – thường biểu diễn theo code 0 đến 8 hoặc phần trăm | (0–100%) hoặc mã code |

---

## ✅ Tóm tắt sử dụng:

| Loại | Thông số quan trọng |
|------|----------------------|
| Dự báo nhiệt độ | `temp`, `dwpt`, `rhum`, `pres` |
| Dự báo mưa | `prcp`, `rhum`, `pres`, `temp`, `coco` |
| Dự báo gió | `wspd`, `wpgt`, `wdir` |
| Dự báo độ nắng | `tsun`, `coco`, `pres`, `rhum` |

---


- Dùng các cột như `temp`, `rhum`, `wspd`, `pres`, `prcp`, `coco` làm **đặc trưng đầu vào (features)**
- Dự báo `temp`, `rhum`, `prcp`… trong 3–5 ngày tới

### Cào dữ liệu từ 2025-01-01 đến 2025-05-01

In [3]:
from datetime import datetime, timedelta
from meteostat import Point, Hourly
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time 
import os   

PROVINCE_LIST_FILE = "danh_sach_tinh_thanh_khong_dau.txt"

# --- Hàm lấy tọa độ từ tên thành phố ---
def get_coordinates(city_name, retries=3, delay=2):
    """
    Sử dụng Geopy để lấy tọa độ (latitude, longitude) từ tên thành phố.
    """
    # Quan trọng: Sử dụng user_agent duy nhất và tuân thủ chính sách sử dụng của Nominatim
    geolocator = Nominatim(user_agent="tide.tantai@gmail.com") # Thay bằng email của bạn hoặc ID duy nhất
    print(f"[{city_name}] Đang tìm tọa độ...")
    for attempt in range(retries):
        try:
            # Thêm quốc gia để tăng độ chính xác cho tên tiếng Việt không dấu
            location = geolocator.geocode(f"{city_name}, Vietnam", timeout=15)
            if location:
                print(f"[{city_name}] Đã tìm thấy tọa độ: ({location.latitude:.4f}, {location.longitude:.4f})")
                return location.latitude, location.longitude
            else:
                # Thử lại không có ", Vietnam" nếu lần đầu thất bại
                print(f"[{city_name}] Không tìm thấy với ', Vietnam', thử lại chỉ với tên...")
                time.sleep(delay / 2) # Delay nhỏ trước khi thử lại
                location = geolocator.geocode(city_name, timeout=15)
                if location:
                    print(f"[{city_name}] Đã tìm thấy tọa độ (lần 2): ({location.latitude:.4f}, {location.longitude:.4f})")
                    return location.latitude, location.longitude
                else:
                    print(f"[{city_name}] Không tìm thấy tọa độ.")
                    return None, None # Trả về None nếu không tìm thấy
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"[{city_name}] Lỗi Geocoding (lần {attempt + 1}/{retries}): {e}. Đang thử lại sau {delay} giây...")
            time.sleep(delay)
        except Exception as e:
            print(f"[{city_name}] Lỗi Geocoding không xác định: {e}")
            return None, None
    print(f"[{city_name}] Không thể lấy tọa độ sau {retries} lần thử.")
    return None, None

# --- Hàm lấy dữ liệu thời tiết LỊCH SỬ THEO GIỜ từ Meteostat ---
def get_hourly_historical_weather(city_name, lat, lon, start_date, end_date):
    """
    Lấy dữ liệu thời tiết lịch sử THEO GIỜ từ Meteostat bằng tọa độ.
    """
    try:
        location = Point(lat, lon)
        start_date_str = start_date.strftime('%Y-%m-%d')
        end_date_str = end_date.strftime('%Y-%m-%d')
        print(f"[{city_name}] Đang lấy dữ liệu Meteostat HOURLY ({lat:.4f}, {lon:.4f}) từ {start_date_str} đến {end_date_str}...")

        data_fetcher = Hourly(location, start_date, end_date)
        df = data_fetcher.fetch()

        if df.empty:
            print(f"[{city_name}] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.")
            return None

        print(f"[{city_name}] Đã lấy thành công {len(df)} bản ghi theo giờ.")
        df['source'] = 'Meteostat'
        df['province'] = city_name # Thêm cột tên tỉnh/thành phố
        return df

    except Exception as e:
        print(f"[{city_name}] Lỗi khi lấy dữ liệu từ Meteostat: {e}")
        return None

# --- Hàm đọc danh sách tỉnh từ file ---
def read_province_list(filename):
    """Đọc danh sách tỉnh từ file text, bỏ qua dòng trống và loại bỏ khoảng trắng thừa."""
    if not os.path.exists(filename):
        print(f"Lỗi: Không tìm thấy file danh sách tỉnh: {filename}")
        return None
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            provinces = [line.strip() for line in f if line.strip()] # Đọc, loại bỏ khoảng trắng, bỏ qua dòng trống
        print(f"Đã đọc được {len(provinces)} tỉnh/thành từ file {filename}.")
        return provinces
    except Exception as e:
        print(f"Lỗi khi đọc file {filename}: {e}")
        return None

# --- Hàm chính để chạy chương trình ---
if __name__ == "__main__":
    provinces = read_province_list(PROVINCE_LIST_FILE)

    if provinces:
        # Xác định khoảng thời gian: 30 ngày gần nhất
        end_date = datetime(2025, 5, 1)
        start_date = datetime(2025, 1, 1)
        start_date_str_file = start_date.strftime('%Y%m%d')
        end_date_str_file = end_date.strftime('%Y%m%d')

        # Tạo thư mục để lưu kết quả nếu chưa có
        output_dir = "weather_data_hourly"
        os.makedirs(output_dir, exist_ok=True)
        print(f"Dữ liệu sẽ được lưu vào thư mục: {output_dir}")

        total_provinces = len(provinces)
        success_count = 0
        fail_coord_count = 0
        fail_fetch_count = 0

        # Lặp qua từng tỉnh trong danh sách
        for i, city_name in enumerate(provinces):
            print(f"\n--- [{i+1}/{total_provinces}] Đang xử lý: {city_name} ---")

            latitude, longitude = get_coordinates(city_name)

            if latitude is None or longitude is None:
                print(f"[{city_name}] Bỏ qua do không lấy được tọa độ.")
                fail_coord_count += 1
                time.sleep(1) # Delay nhỏ trước khi xử lý tỉnh tiếp theo
                continue # Chuyển sang tỉnh tiếp theo

            # Delay giữa các lần gọi Geocoding để tránh bị block
            time.sleep(1.5) # Chờ 1.5 giây

            historical_weather_hourly = get_hourly_historical_weather(city_name, latitude, longitude, start_date, end_date)

            if historical_weather_hourly is not None and not historical_weather_hourly.empty:
                # Tạo tên file động và đường dẫn đầy đủ
                safe_city_name = "".join(c if c.isalnum() else "_" for c in city_name.replace(" ", "_"))
                filename = f"hourly_weather_{safe_city_name}_{start_date_str_file}_to_{end_date_str_file}.csv"
                filepath = os.path.join(output_dir, filename)

                try:
                    historical_weather_hourly.to_csv(filepath, index=True)
                    print(f"[{city_name}] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: {filepath}")
                    success_count += 1
                except Exception as e:
                    print(f"[{city_name}] Lỗi khi lưu file CSV '{filepath}': {e}")
                    # Có thể coi đây là một dạng lỗi fetch nếu lưu thất bại
                    fail_fetch_count +=1

            else:
                print(f"[{city_name}] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.")
                fail_fetch_count += 1

            # Delay giữa các lần xử lý tỉnh để tránh quá tải Meteostat (nếu cần)
            time.sleep(0.5) # Chờ nửa giây

        print("\n--- TỔNG KẾT ---")
        print(f"Tổng số tỉnh/thành cần xử lý: {total_provinces}")
        print(f"Số lượng xử lý và lưu thành công: {success_count}")
        print(f"Số lượng lỗi không tìm thấy tọa độ: {fail_coord_count}")
        print(f"Số lượng lỗi không lấy được/lưu dữ liệu thời tiết: {fail_fetch_count}")
        print("Chương trình hoàn tất.")

    else:
        print("Không có danh sách tỉnh để xử lý. Chương trình kết thúc.")

Đã đọc được 63 tỉnh/thành từ file danh_sach_tinh_thanh_khong_dau.txt.
Dữ liệu sẽ được lưu vào thư mục: weather_data_hourly

--- [1/63] Đang xử lý: An Giang ---
[An Giang] Đang tìm tọa độ...
[An Giang] Đã tìm thấy tọa độ: (10.5392, 105.2313)
[An Giang] Đang lấy dữ liệu Meteostat HOURLY (10.5392, 105.2313) từ 2025-01-01 đến 2025-05-01...
[An Giang] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[An Giang] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [2/63] Đang xử lý: Ba Ria - Vung Tau ---
[Ba Ria - Vung Tau] Đang tìm tọa độ...
[Ba Ria - Vung Tau] Đã tìm thấy tọa độ: (10.5739, 107.3284)
[Ba Ria - Vung Tau] Đang lấy dữ liệu Meteostat HOURLY (10.5739, 107.3284) từ 2025-01-01 đến 2025-05-01...
[Ba Ria - Vung Tau] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Ba Ria - Vung Tau] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [3/63] Đang xử lý: Bac Giang ---
[Bac Giang] Đang tìm tọa độ...
[Bac Giang] Đã tìm thấy tọa độ: (21.3170, 106.


--- [12/63] Đang xử lý: Ca Mau ---
[Ca Mau] Đang tìm tọa độ...
[Ca Mau] Đã tìm thấy tọa độ: (9.0180, 105.0870)
[Ca Mau] Đang lấy dữ liệu Meteostat HOURLY (9.0180, 105.0870) từ 2025-01-01 đến 2025-05-01...
[Ca Mau] Đã lấy thành công 2881 bản ghi theo giờ.
[Ca Mau] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Ca_Mau_20250101_to_20250501.csv



--- [13/63] Đang xử lý: Can Tho ---
[Can Tho] Đang tìm tọa độ...
[Can Tho] Đã tìm thấy tọa độ: (10.0365, 105.7876)
[Can Tho] Đang lấy dữ liệu Meteostat HOURLY (10.0365, 105.7876) từ 2025-01-01 đến 2025-05-01...
[Can Tho] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Can Tho] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [14/63] Đang xử lý: Cao Bang ---
[Cao Bang] Đang tìm tọa độ...
[Cao Bang] Đã tìm thấy tọa độ: (22.7427, 106.1061)
[Cao Bang] Đang lấy dữ liệu Meteostat HOURLY (22.7427, 106.1061) từ 2025-01-01 đến 2025-05-01...
[Cao Bang] Đã lấy thành công 2881 bản ghi theo giờ.
[Cao Bang] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Cao_Bang_20250101_to_20250501.csv



--- [15/63] Đang xử lý: Da Nang ---
[Da Nang] Đang tìm tọa độ...
[Da Nang] Đã tìm thấy tọa độ: (16.0680, 108.2120)
[Da Nang] Đang lấy dữ liệu Meteostat HOURLY (16.0680, 108.2120) từ 2025-01-01 đến 2025-05-01...
[Da Nang] Đã lấy thành công 2881 bản ghi theo giờ.
[Da Nang] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Da_Nang_20250101_to_20250501.csv



--- [16/63] Đang xử lý: Dak Lak ---
[Dak Lak] Đang tìm tọa độ...
[Dak Lak] Đã tìm thấy tọa độ: (12.6922, 108.0637)
[Dak Lak] Đang lấy dữ liệu Meteostat HOURLY (12.6922, 108.0637) từ 2025-01-01 đến 2025-05-01...
[Dak Lak] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Dak Lak] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [17/63] Đang xử lý: Dak Nong ---
[Dak Nong] Đang tìm tọa độ...
[Dak Nong] Đã tìm thấy tọa độ: (12.2819, 107.7302)
[Dak Nong] Đang lấy dữ liệu Meteostat HOURLY (12.2819, 107.7302) từ 2025-01-01 đến 2025-05-01...
[Dak Nong] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Dak Nong] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [18/63] Đang xử lý: Dien Bien ---
[Dien Bien] Đang tìm tọa độ...
[Dien Bien] Đã tìm thấy tọa độ: (21.6547, 103.2169)
[Dien Bien] Đang lấy dữ liệu Meteostat HOURLY (21.6547, 103.2169) từ 2025-01-01 đến 2025-05-01...
[Dien Bien] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Dien Bien

...

[Quang Binh] Đã lấy thành công 2881 bản ghi theo giờ.
[Quang Binh] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Quang_Binh_20250101_to_20250501.csv

--- [46/63] Đang xử lý: Quang Nam ---
[Quang Nam] Đang tìm tọa độ...
[Quang Nam] Đã tìm thấy tọa độ: (15.5762, 108.0527)
[Quang Nam] Đang lấy dữ liệu Meteostat HOURLY (15.5762, 108.0527) từ 2025-01-01 đến 2025-05-01...
[Quang Nam] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Quang Nam] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [47/63] Đang xử lý: Quang Ngai ---
[Quang Ngai] Đang tìm tọa độ...
[Quang Ngai] Đã tìm thấy tọa độ: (15.1210, 108.7814)
[Quang Ngai] Đang lấy dữ liệu Meteostat HOURLY (15.1210, 108.7814) từ 2025-01-01 đến 2025-05-01...
[Quang Ngai] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Quang Ngai] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [48/63] Đang xử lý: Quang Ninh ---
[Quang Ninh] Đang tìm tọa độ...
[Quang Ninh] Đã tìm thấy t

[Son La] Đã lấy thành công 2881 bản ghi theo giờ.
[Son La] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Son_La_20250101_to_20250501.csv

--- [52/63] Đang xử lý: Tay Ninh ---
[Tay Ninh] Đang tìm tọa độ...
[Tay Ninh] Đã tìm thấy tọa độ: (11.4019, 106.1627)
[Tay Ninh] Đang lấy dữ liệu Meteostat HOURLY (11.4019, 106.1627) từ 2025-01-01 đến 2025-05-01...
[Tay Ninh] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Tay Ninh] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [53/63] Đang xử lý: Thai Binh ---
[Thai Binh] Đang tìm tọa độ...
[Thai Binh] Đã tìm thấy tọa độ: (20.4493, 106.3425)
[Thai Binh] Đang lấy dữ liệu Meteostat HOURLY (20.4493, 106.3425) từ 2025-01-01 đến 2025-05-01...
[Thai Binh] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Thai Binh] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [54/63] Đang xử lý: Thai Nguyen ---
[Thai Nguyen] Đang tìm tọa độ...
[Thai Nguyen] Đã tìm thấy tọa độ: (21.6499, 105.

[Thanh Hoa] Đã lấy thành công 2881 bản ghi theo giờ.
[Thanh Hoa] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Thanh_Hoa_20250101_to_20250501.csv

--- [56/63] Đang xử lý: Thanh pho Ho Chi Minh ---
[Thanh pho Ho Chi Minh] Đang tìm tọa độ...
[Thanh pho Ho Chi Minh] Đã tìm thấy tọa độ: (10.7764, 106.7011)
[Thanh pho Ho Chi Minh] Đang lấy dữ liệu Meteostat HOURLY (10.7764, 106.7011) từ 2025-01-01 đến 2025-05-01...


[Thanh pho Ho Chi Minh] Đã lấy thành công 2881 bản ghi theo giờ.
[Thanh pho Ho Chi Minh] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Thanh_pho_Ho_Chi_Minh_20250101_to_20250501.csv

--- [57/63] Đang xử lý: Thua Thien Hue ---
[Thua Thien Hue] Đang tìm tọa độ...
[Thua Thien Hue] Đã tìm thấy tọa độ: (16.4639, 107.5863)
[Thua Thien Hue] Đang lấy dữ liệu Meteostat HOURLY (16.4639, 107.5863) từ 2025-01-01 đến 2025-05-01...


[Thua Thien Hue] Đã lấy thành công 2881 bản ghi theo giờ.
[Thua Thien Hue] ĐÃ LƯU DỮ LIỆU THÀNH CÔNG vào file: weather_data_hourly\hourly_weather_Thua_Thien_Hue_20250101_to_20250501.csv

--- [58/63] Đang xử lý: Tien Giang ---
[Tien Giang] Đang tìm tọa độ...
[Tien Giang] Đã tìm thấy tọa độ: (10.4030, 106.3616)
[Tien Giang] Đang lấy dữ liệu Meteostat HOURLY (10.4030, 106.3616) từ 2025-01-01 đến 2025-05-01...
[Tien Giang] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Tien Giang] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [59/63] Đang xử lý: Tra Vinh ---
[Tra Vinh] Đang tìm tọa độ...
[Tra Vinh] Đã tìm thấy tọa độ: (9.8038, 106.3257)
[Tra Vinh] Đang lấy dữ liệu Meteostat HOURLY (9.8038, 106.3257) từ 2025-01-01 đến 2025-05-01...
[Tra Vinh] Không tìm thấy dữ liệu lịch sử theo giờ từ Meteostat.
[Tra Vinh] Bỏ qua do không lấy được dữ liệu thời tiết hoặc dữ liệu trống.

--- [60/63] Đang xử lý: Tuyen Quang ---
[Tuyen Quang] Đang tìm tọa độ...
[Tuyen Quang] Đã tìm